## First we need all necessary imports

In [1]:
import os
import numpy as np
import flopy.modflow as mf
import shutil
import geojson
import pyproj

* The modflow-files will be stored in './workspace' 
* Each calculation will delete the files

In [2]:
workspace = os.path.join('workspace')

if os.path.exists(workspace):
    shutil.rmtree(workspace)
    
if not os.path.exists(workspace):
    os.makedirs(workspace)


In [3]:
import json
f = open('configuration.json')
configuration = json.load(f)
f.close()

modflow_data = configuration['data']['mf']
modflow_package_list = modflow_data['packages']

modflow_package_list

['mf', 'bas', 'dis', 'lpf', 'pcg', 'oc', 'chd', 'lak']

In [4]:
from MfPackages import MfAdapter
mf_data_obj = modflow_data['mf']
mf_data_obj['modelname'] = 'lak_test'
mf_data_obj['exe_name'] = '../modflow/mf2005'
mf_data_obj['model_ws'] = workspace
model = MfAdapter(mf_data_obj).get_package()

In [5]:
from MfPackages import DisAdapter
dis = DisAdapter(modflow_data['dis']).get_package(model)

In [6]:
from MfPackages import BasAdapter
bas = BasAdapter(modflow_data['bas']).get_package(model)

In [7]:
from MfPackages import ChdAdapter
chd = ChdAdapter(modflow_data['chd']).get_package(model)

In [8]:
from MfPackages import LakAdapter
lak = LakAdapter(modflow_data['lak']).get_package(model)

In [9]:
from MfPackages import LpfAdapter
lpf = LpfAdapter(modflow_data['lpf']).get_package(model)

In [10]:
from MfPackages import PcgAdapter
pcg = PcgAdapter(modflow_data['pcg']).get_package(model)

In [11]:
from MfPackages import OcAdapter
oc = OcAdapter(modflow_data['oc']).get_package(model)

### Calculate the model

In [12]:
model.write_input()
model.run_model()

FloPy is using the following executable to run the model: ../../modflow/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: lak_test.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/08/24 11:53:34

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/08/24 11:53:34
 Elapsed run time:  0.002 Seconds

  Normal termination of simulation


(True, [])

### Read calculated data

In [13]:
import flopy.utils as fu

hds = fu.HeadFile(os.path.join(workspace, 'lak_test.hds'))
h = hds.get_data(kstpkper=(0,0))
h

array([[[450.     , 449.55933, 449.10083, 448.60574, 448.06143,
         447.5    , 446.93854, 446.39426, 445.89917, 445.44067,
         445.     ],
        [450.     , 449.6005 , 449.18555, 448.71964, 448.1012 ,
         447.5    , 446.89877, 446.28036, 445.81445, 445.3995 ,
         445.     ],
        [450.     , 449.67746, 449.38818, 449.18646, 449.     ,
         449.     , 449.     , 445.81354, 445.61182, 445.32254,
         445.     ],
        [450.     , 449.67746, 449.38818, 449.18646, 449.     ,
         449.     , 449.     , 445.81354, 445.61182, 445.32254,
         445.     ],
        [450.     , 449.6005 , 449.18555, 448.71964, 448.1012 ,
         447.5    , 446.89877, 446.28036, 445.81445, 445.3995 ,
         445.     ],
        [450.     , 449.55933, 449.10083, 448.60574, 448.06143,
         447.5    , 446.93854, 446.39426, 445.89917, 445.44067,
         445.     ]]], dtype=float32)